In [1]:
import os
import torch
import numpy as np
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import functional as F
from tqdm import tqdm  # For progress tracking

/home/tanush/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### For Testing on multiple images in a folder

In [2]:
# Define paths
image_folder = "./Dataset/test/images"
output_folder = "output"

In [3]:
def get_model(num_keypoints, weights_path=None):
    anchor_generator = AnchorGenerator(
        sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0)
    )
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(
        pretrained=False,
        pretrained_backbone=True,
        num_keypoints=num_keypoints,
        num_classes=2,  # Background + 1 object class
        rpn_anchor_generator=anchor_generator,
    )

    if weights_path:
        model.load_state_dict(torch.load(weights_path))  # Load trained weights
        print("Model weights loaded successfully!")
    return model

# Define the number of keypoints and the path to the saved weights
num_keypoints = 8
weights_path = "UBS.pth"

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model(num_keypoints, weights_path).to(device)
model.eval()  # Set model to evaluation mode

def visualize_best_prediction(image_path, prediction):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extract bounding boxes, keypoints, and scores
    boxes = prediction['boxes']
    keypoints = prediction['keypoints']
    scores = prediction['scores']

    # Find the index of the highest confidence score
    best_idx = scores.argmax()
    best_box = boxes[best_idx].int().tolist()   # Coordinates of the best box
    best_keypoints = keypoints[best_idx]       # Keypoints of the best box

    # Draw the highest confidence bounding box
    cv2.rectangle(image, (best_box[0], best_box[1]), (best_box[2], best_box[3]), color=(0, 255, 0), thickness=2)

    # Draw keypoints for the best box
    for kp in best_keypoints:
        kp_x, kp_y, visibility = map(int, kp)
        if visibility > 0:  # Only draw visible keypoints
            cv2.circle(image, (kp_x, kp_y), radius=3, color=(255, 0, 0), thickness=-1)

    # Display the image
    plt.imshow(image)
    plt.axis("off")
    plt.show()

def preprocess_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to a tensor
    image_tensor = F.to_tensor(image)  # Converts [H, W, C] to [C, H, W] and normalizes to [0, 1]
    return image_tensor

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def visualize_and_save(image_path, predictions, output_path):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extract the best prediction (highest confidence score)
    boxes = predictions['boxes'].cpu().numpy()
    keypoints = predictions['keypoints'].cpu().numpy()
    scores = predictions['scores'].cpu().numpy()

    if len(scores) > 0:
        best_idx = np.argmax(scores)  # Index of the highest confidence score
        best_box = boxes[best_idx].astype(int).tolist()
        best_keypoints = keypoints[best_idx].astype(int).tolist()

        # Draw the best bounding box
        cv2.rectangle(image, (best_box[0], best_box[1]), (best_box[2], best_box[3]), color=(0, 255, 0), thickness=2)

        # Draw keypoints
        for kp in best_keypoints:
            kp_x, kp_y, visibility = kp
            if visibility > 0:  # Only draw visible keypoints
                cv2.circle(image, (kp_x, kp_y), radius=3, color=(255, 0, 0), thickness=-1)

        # Save the image with bounding boxes and keypoints
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert back to BGR for saving with OpenCV
        cv2.imwrite(output_path, image)

# Process all images in the folder
for image_filename in tqdm(os.listdir(image_folder)):
    if not image_filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue  # Skip non-image files

    # Load and preprocess the image
    image_path = os.path.join(image_folder, image_filename)
    image_tensor = preprocess_image(image_path)
    image_tensor = image_tensor.to(device)
    image_batch = image_tensor.unsqueeze(0)  # Add batch dimension

    # Perform prediction
    with torch.no_grad():
        predictions = model(image_batch)

    # Extract the output file path (same as input, but in output folder)
    output_image_path = os.path.join(output_folder, image_filename)

    # Visualize and save the best prediction
    visualize_and_save(image_path, predictions[0], output_image_path)

print(f"Predictions saved to: {output_folder}")

/home/tanush/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tanush/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/tanush/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and may be removed in the future, please use 'weights_backbone' instead.
  warnings.warn(
/home/tanush/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights_backbone' are deprecated since 0.13 and may be removed in the future. Th

Model weights loaded successfully!


100%|██████████| 20/20 [00:06<00:00,  3.27it/s]

Predictions saved to: output
